# Julia DEM SVD

In [1]:
import Pkg
Pkg.add("Flux")
Pkg.add("Plots")
Pkg.add("NetCDF")
Pkg.add("TSVD")
Pkg.add("IterTools")
Pkg.add("Statistics")
Pkg.add("Compat")
Pkg.add("LinearAlgebra")
Pkg.add("Glob")
using Flux
using Flux: train!
using Plots
using TSVD
using IterTools
using IterTools: ncycle 
using Statistics
using LinearAlgebra
using Compat
using Glob
using NetCDF

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [e66e0078] ↑ CompilerSupportLibraries_jll v0.5.2+0 ⇒ v1.0.1+0
Precompiling project...
  ✓ RangeArrays
  ✓ FunctionWrappers
  ✓ Calculus
  ✓ Combinatorics
  ✓ ScientificTypesBase
  ✓ LaTeXStrings
  ✓ SentinelArrays
  ✓ StructTypes
  ✓ PositiveFactorizations
  ✓ Glob
  ✓ IteratorInterfaceExtensions
  ✓ ExprTools
  ✓ PDMats
  ✓ WorkerUtilities
  ✓ TensorCore
  ✓ StatsAPI
  ✓ Contour
  ✓ CEnum
  ✓ InplaceOps
  ✓ NaturalSort
  ✓ InitialValues
  ✓ Pipe
  ✓ UnPack
  ✓ OpenLibm_jll
  ✓ Measures
  ✓ InverseFunctions
  ✓ BufferedStreams
  ✓ CommonSolve
  ✓ PrettyPrint
  ✓ ArgCheck
  ✓ FillArrays
  ✓ FixedPointNumbers
  ✓ WoodburyMatrices
  ✓ CompilerSupportLibraries_jll
  ✓ ShowCases
  ✓ IterTools
  ✓ Compat
  ✓ Grisu
  ✓ OrderedCollections
  ✓ InternedStrings
  ✓ StringManipulation
  ✓ Ext

  ✓ DiskArrays
  ✓ TransmuteDims
  ✓ SplittablesBase
  ✓ Fontconfig_jll
  ✓ Pickle
  ✓ Roots
  ✓ NetCDF_jll
  ✓ Wayland_jll
  ✓ Gettext_jll
  ✓ XSLT_jll
  ✓ HTTP
  ✓ TableOperations
  ✓ StructArrays
  ✓ HDF5
  ✓ BangBang
  ✓ RecursiveArrayTools
  ✓ FiniteDiff
  ✓ PrettyTables
  ✓ NearestNeighbors
  ✓ JLD2
  ✓ LogExpFunctions
  ✓ Interpolations
  ✓ Graphics
  ✓ GPUArrays
  ✓ FFTW
  ✓ Widgets
  ✓ Arpack
  ✓ MosaicViews
  ✓ OneHotArrays
  ✓ PyPlot
  ✓ NetCDF
  ✓ GPUCompiler
  ✓ Glib_jll
  ✓ Xorg_libxcb_jll
  ✓ CSV
  ✓ DataDeps
  ✓ MicroCollections
  ✓ MAT
  ✓ NCDatasets
  ✓ StatsBase
  ✓ TensorCast
  ✓ ChainRules
  ✓ PSIS
  ✓ SciMLBase
  ✓ Xorg_xcb_util_jll
  ✓ Xorg_libX11_jll
  ✓ SpecialFunctions
  ✓ Clustering
  ✓ MultivariateStats
  ✓ Transducers
  ✓ Xorg_xcb_util_image_jll
  ✓ Xorg_xcb_util_keysyms_jll
  ✓ Xorg_xcb_util_renderutil_jll
  ✓ Optimization
  ✓ Xorg_xcb_util_wm_jll
  ✓ Xorg_libXrender_jll
  ✓ Xorg_libXext_jll
  ✓ Metal
  ✓ Xorg_libXfixes_jll
  ✓ Xorg_libxkbfile_jll
  ✓ Diff

In [11]:
training_files = glob("usurf_ex_gris_g1800m_v5_RAGIS_id_*.nc")

nf = length(training_files)
d = ncread(training_files[1], "usurf")
nx, ny, nt = size(d)
x = ncread(training_files[1], "x")
y = ncread(training_files[1], "y")

Data = zeros(nf * nt, nx*ny)
for (k, training_file) in enumerate(training_files)
    println(training_file)
    d = ncread(training_file, "usurf")
    data = transpose(reshape(d, nx * ny, nt))
    Data[(k - 1 ) * nt + 1:k * nt, :] = data
end


obs_file ="aerodem_1978_1987_mean_g1800m.nc"
obs = ncread(obs_file, "surface")
nx_obs, ny_obs = size(obs)

R = reshape(obs, nx * ny);
I = findall(x->x!=-9999, R);

println(I)

usurf_ex_gris_g1800m_v5_RAGIS_id_0_1975-1-1_1980-1-1.nc
usurf_ex_gris_g1800m_v5_RAGIS_id_1_1975-1-1_1980-1-1.nc


[8315, 8320, 8321, 8324, 8331, 8332, 8333, 9201, 9202, 9204, 9207, 9210, 9213, 9214, 9231, 9232, 10090, 10091, 10092, 10093, 10094, 10095, 10097, 10963, 10964, 10967, 10969, 10970, 10971, 10972, 10973, 10974, 10975, 10976, 10979, 11843, 11844, 11846, 11847, 11848, 11849, 11850, 11851, 11852, 11853, 11854, 11855, 11857, 11858, 11859, 11860, 11870, 12724, 12726, 12727, 12728, 12729, 12730, 12731, 12732, 12733, 12734, 12735, 12736, 12737, 12738, 12739, 12740, 12741, 12742, 12743, 12744, 12745, 12746, 12749, 12750, 12753, 13588, 13597, 13599, 13601, 13603, 13604, 13605, 13606, 13607, 13608, 13609, 13610, 13611, 13612, 13613, 13614, 13615, 13616, 13617, 13618, 13619, 13620, 13621, 13622, 13623, 13624, 13625, 13626, 13627, 13628, 13629, 13630, 13631, 13632, 13633, 13634, 13636, 14456, 14457, 14458, 14460, 14461, 14464, 14469, 14471, 14479, 14481, 14482, 14483, 14486, 14487, 14488, 14489, 14490, 14491, 14492, 14493, 14494, 14495, 14496, 14497, 14498, 14499, 14500, 14501, 14502, 14503, 14504, 

 36480, 36481, 36482, 36483, 36484, 36485, 36486, 36487, 36488, 36489, 36490, 36491, 36492, 36493, 36494, 36495, 36496, 36497, 36498, 36499, 36500, 36501, 36502, 36503, 36504, 36507, 36508, 36509, 36510, 36511, 36512, 36513, 36514, 36515, 36516, 36517, 36518, 36519, 36520, 36521, 36522, 36523, 36524, 36525, 36526, 36527, 36528, 37320, 37321, 37322, 37323, 37324, 37325, 37326, 37327, 37328, 37329, 37330, 37331, 37332, 37333, 37334, 37335, 37336, 37337, 37338, 37339, 37340, 37341, 37342, 37343, 37344, 37345, 37346, 37347, 37348, 37349, 37350, 37351, 37352, 37353, 37354, 37355, 37356, 37357, 37358, 37359, 37360, 37361, 37362, 37363, 37364, 37365, 37366, 37367, 37368, 37369, 37370, 37371, 37372, 37373, 37374, 37375, 37376, 37377, 37378, 37379, 37380, 37382, 37383, 37384, 37386, 37387, 37388, 37389, 37390, 37391, 37392, 37393, 37394, 37395, 37396, 37397, 37398, 37399, 37400, 37401, 37402, 37403, 37404, 37405, 37406, 37407, 37408, 38200, 38201, 38202, 38203, 38204, 38205, 38206, 38207, 38208

, 50548, 50549, 50550, 50551, 50552, 50553, 50554, 50555, 50556, 50557, 50558, 50559, 50560, 50561, 50562, 50563, 50564, 50565, 50566, 50567, 50568, 50569, 50570, 50571, 50572, 50573, 50574, 50575, 50576, 50577, 50578, 50579, 50580, 50581, 50582, 50583, 50584, 50585, 50586, 50587, 50588, 50589, 50590, 50591, 50592, 50593, 50594, 50595, 50596, 50597, 50598, 50599, 50600, 50601, 50602, 50603, 50604, 50605, 50606, 50607, 50608, 51381, 51382, 51383, 51386, 51387, 51388, 51389, 51390, 51391, 51392, 51393, 51394, 51395, 51396, 51397, 51398, 51399, 51400, 51401, 51402, 51403, 51404, 51405, 51406, 51407, 51408, 51409, 51410, 51411, 51412, 51413, 51414, 51415, 51416, 51417, 51418, 51419, 51420, 51421, 51422, 51423, 51424, 51425, 51426, 51427, 51428, 51429, 51430, 51431, 51432, 51433, 51434, 51435, 51436, 51437, 51438, 51439, 51440, 51441, 51442, 51443, 51444, 51445, 51446, 51447, 51448, 51449, 51450, 51451, 51452, 51453, 51454, 51455, 51456, 51457, 51458, 51459, 51460, 51461, 51462, 51463, 5146

 64618, 64619, 64620, 64621, 64622, 64623, 64624, 64625, 64626, 64627, 64628, 64629, 64630, 64631, 64632, 64633, 64634, 64635, 64636, 64641, 64642, 64643, 64648, 64649, 64650, 64651, 64652, 64653, 64654, 64655, 64656, 64657, 64658, 64659, 64660, 64661, 64662, 64663, 64664, 64665, 64666, 64667, 64668, 64669, 64670, 64671, 64672, 64673, 64674, 64675, 64676, 64679, 64680, 64685, 64686, 64687, 64688, 64689, 64690, 64691, 64692, 64693, 64694, 64695, 64696, 64697, 64698, 64699, 64700, 65399, 65400, 65401, 65402, 65403, 65404, 65405, 65406, 65407, 65408, 65409, 65410, 65411, 65412, 65413, 65414, 65415, 65416, 65417, 65419, 65420, 65421, 65422, 65423, 65424, 65425, 65427, 65428, 65429, 65434, 65435, 65436, 65437, 65438, 65439, 65440, 65441, 65442, 65443, 65444, 65445, 65446, 65447, 65448, 65449, 65450, 65451, 65452, 65453, 65454, 65455, 65456, 65457, 65458, 65459, 65460, 65461, 65462, 65463, 65464, 65465, 65466, 65468, 65469, 65470, 65471, 65472, 65473, 65475, 65476, 65477, 65478, 65479, 65480

...

424419, 424420, 424421, 424422, 424423, 424424, 424425, 424426, 424427, 424428, 424429, 424430, 424431, 424432, 424433, 424718, 424719, 424720, 424721, 424722, 424723, 424724, 424725, 424733, 424734, 424735, 424736, 424737, 424738, 424739, 424740, 424741, 424742, 424743, 424744, 424745, 424746, 424747, 424748, 424749, 424750, 424751, 424752, 424753, 424754, 424755, 424756, 424757, 424758, 424759, 424760, 424761, 424762, 424763, 424764, 424765, 424766, 424767, 424768, 424769, 424770, 424771, 424773, 424783, 424784, 424785, 424789, 424790, 424791, 424792, 424793, 424794, 424795, 424796, 424797, 424798, 424799, 424800, 424801, 424802, 424803, 424804, 424805, 424806, 424807, 424808, 424809, 424810, 424811, 424812, 424813, 424814, 425191, 425192, 425193, 425203, 425204, 425205, 425206, 425207, 425208, 425209, 425210, 425211, 425212, 425213, 425214, 425215, 425216, 425217, 425218, 425219, 425220, 425221, 425222, 425223, 425224, 425225, 425226, 425227, 425228, 425229, 425230, 425231, 425232, 

 429655, 429656, 429657, 429658, 429659, 429660, 429661, 429662, 429663, 429664, 429665, 429666, 429667, 429668, 429669, 429670, 429671, 429672, 429673, 429674, 429675, 429676, 429677, 429678, 429679, 429680, 429681, 429682, 429683, 429684, 429685, 429686, 429687, 429688, 429689, 429690, 429691, 429692, 429693, 429694, 429695, 429696, 429697, 429698, 429699, 429700, 429701, 429702, 429703, 429704, 429705, 429706, 429707, 429708, 429709, 429710, 429711, 429712, 429713, 430017, 430018, 430019, 430020, 430021, 430022, 430023, 430024, 430025, 430026, 430027, 430028, 430029, 430030, 430031, 430032, 430033, 430034, 430035, 430036, 430037, 430038, 430039, 430040, 430041, 430042, 430043, 430044, 430046, 430047, 430048, 430049, 430050, 430051, 430052, 430053, 430054, 430055, 430056, 430057, 430058, 430059, 430060, 430061, 430064, 430065, 430066, 430067, 430068, 430069, 430070, 430071, 430072, 430073, 430074, 430075, 430076, 430077, 430078, 430079, 430080, 430081, 430082, 430083, 430084, 430085,

 435789, 435790, 435791, 435792, 435793, 435794, 435795, 435796, 435797, 435798, 435799, 435800, 435801, 435802, 435803, 435804, 435805, 435806, 435807, 435808, 435809, 435810, 435811, 435812, 435813, 435814, 435815, 435816, 435817, 435818, 435819, 435820, 435821, 435822, 435823, 435824, 435825, 435826, 435827, 435828, 435829, 435830, 435831, 435832, 435833, 435834, 435835, 435836, 435837, 435838, 435839, 435840, 435841, 435842, 435843, 435844, 435845, 435846, 435847, 435848, 435849, 435850, 435851, 435852, 435853, 435854, 435855, 435856, 435857, 435858, 435859, 435860, 435861, 435862, 435863, 435864, 435865, 435866, 435867, 435868, 435869, 435870, 435871, 435872, 435873, 435874, 436182, 436183, 436184, 436185, 436186, 436187, 436188, 436189, 436190, 436191, 436192, 436193, 436194, 436195, 436196, 436201, 436202, 436203, 436204, 436206, 436208, 436210, 436211, 436212, 436213, 436214, 436215, 436219, 436220, 436221, 436222, 436223, 436224, 436225, 436226, 436227, 436231, 436232, 436233,

Excessive output truncated after 524288 bytes.

, 441118, 441119, 441120, 441121, 441122, 441123, 441124, 441125, 441126, 441129, 441130, 441131, 441132, 441133, 441134, 441135, 441136, 441137, 441138, 441139, 441140, 441141, 441142, 441143, 441144, 441145, 441146, 441147, 441148, 441149, 441150, 441151, 441152, 441153, 441154, 441155, 441465, 441466, 441467, 441468, 441469, 441470, 441471, 441472, 441473, 441474, 441475, 441477, 441478, 441482, 441484, 441485, 441486, 441488, 441489, 441490, 441491, 441499, 441500, 441501, 441502, 441503, 441504, 441505, 441507, 441508, 441509, 441510, 441511, 441513, 441514, 441515, 441516, 441517, 441518, 441519, 441520, 441521, 441522, 441523, 441524, 441525, 441526, 441527, 441528, 441529, 441530, 441531, 441532, 441533, 441534, 441535, 441536, 441537, 441538, 441539, 441540, 441541, 441542, 441543, 441922, 441923, 441924, 441925, 441926, 441927, 441928, 441929, 441930, 441931, 441932, 441933, 441934, 441935, 441936, 441937, 441938, 441939, 441940, 441941, 441942, 441943, 441944, 441945, 441946

In [15]:
q = 10

Data_mean = mean(Data, dims=1)
Data_scaled = Data .- Data_mean
U, S, V = tsvd(Data_scaled, q)
println("dim V = ", size(V))
println("dim S = ", size(S))
println("dim U = ", size(U))
println("dim R = ", size(R))

SV = transpose(V * diagm(S))

println("SV= ",size(SV))

x_train = SV[:, I]
_, n_obs = size(x_train)
y_train = reshape(R[I], n_obs)

dim V = (1337600, 10)
dim S = (10,)
dim U = (120, 10)
dim R = (1337600,)
SV= (10, 1337600)


273500-element Vector{Float32}:
 324.86
  -1.4764
  -1.4764
   1.6914375
  33.279922
  53.697407
  57.06921
   8.879
  12.82
  33.603035
  27.7722
  39.7888
  50.251762
   ⋮
  20.488077
  22.618904
  20.447895
  18.69587
  19.13604
  20.925987
  21.561682
  20.636114
  23.372412
  58.2486
  56.1275
 119.44578

In [16]:
n_epochs = 100

for K in [0 1e4 1e5 1e6 1e7]
    println(K)
    model = Dense(q, 1)
    ps = Flux.params(model)
    penalty() = K * mean(abs2, model.weight)
    loss(x, y) = Flux.Losses.mse(model(x), y) + penalty()
    opt = Adam(0.1)

    loss_0 = loss(x_train, y_train)
    println("Initial loss: $loss_0")

    data = [(x_train, y_train)]

    train_loader = Flux.DataLoader((x_train, y_train), batchsize = 128, shuffle = true)
    Flux.train!(loss, ps, ncycle(train_loader, n_epochs), opt)

    loss_e = loss(x_train, y_train)
    println("Final loss: $loss_e")

    M =  reshape(model.weight * SV + Data_mean, nx, ny)
    M[findall(x->x<0, M)] .= 0

    Ms = copy(reshape(M, nx, ny, 1))

    filename = "dem_ragis_q_$(q)_K_$(K).nc"
    varname  = "usurf"
    attribs  = Dict("units"   => "m",
                    "data_min" => 0.0)

    nccreate(filename, varname, "x", x, "y", y, "time", 1, atts=attribs)
    ncwrite(Ms, filename, varname)
end

0.0


LoadError: DimensionMismatch: loss function expects size(ŷ) = (1, 273500) to match size(y) = (273500,)

In [21]:
heatmap(x, y, transpose(M))



LoadError: UndefVarError: M not defined

In [20]:
    model = Dense(q, 1)



Dense(10 => 1)      # 11 parameters

In [21]:
model(x_train)

1×273500 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [22]:
x_train

10×273500 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0